## Sim Demo

### Imports

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from env.imports import *

In [8]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run) 

<module 'sim.sim_run' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/sim/sim_run.py'>

#### Check job specs

In [4]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 52.2%
RAM Usage: 35.3%
Available RAM: 244.1G
Total RAM: 377.1G
52.4G


## Wandb API Query <a id="sims"></a>

In [5]:
relevant info about wandb

https://wandb.ai/alexander-ratzan-new-york-university/gx2conn?nw=nwuserasratzan
https://wandb.ai/asratzan

import wandb
api = wandb.Api()

run = api.run("alexander-ratzan-new-york-university/gx2conn/<run_id>")
run.config["key"] = updated_value
run.update()

import wandb
api = wandb.Api()

# run is specified by <entity>/<project>/<run_id>
run = api.run("alexander-ratzan-new-york-university/gx2conn/<run_id>")

# save the metrics for the run to a csv file
metrics_dataframe = run.history()
metrics_dataframe.to_csv("metrics.csv")

import wandb
api = wandb.Api()

run = api.run("alexander-ratzan-new-york-university/gx2conn/<run_id>")
if run.state == "finished":
    for i, row in run.history().iterrows():
      print(row["_timestamp"], row["accuracy"])

import wandb
api = wandb.Api()

run = api.run("alexander-ratzan-new-york-university/gx2conn/<run_id>")
history = run.scan_history()
losses = [row["loss"] for row in history]

import wandb
api = wandb.Api()

sweep = api.sweep("alexander-ratzan-new-york-university/gx2conn/<sweep_id>")
runs = sorted(sweep.runs,
  key=lambda run: run.summary.get("val_acc", 0), reverse=True)
val_acc = runs[0].summary.get("val_acc", 0)
print(f"Best run {runs[0].name} with {val_acc}% validation accuracy")

runs[0].file("model.h5").download(replace=True)
print("Best model saved to model-best.h5")

Project visibility
Team
Last active
7/11/2025, 5:46:21 PM
Contributors
1 user
Total runs
20270
Total compute
111 days



MODEL_CLASSES = {
    'cge': CGEModel,
    'gaussian_kernel': GaussianKernelModel,
    'exponential_decay': ExponentialDecayModel,
    'bilinear_lowrank': BilinearLowRank,
    'bilinear_CM': BilinearCM,
    'pls_twostep': PLSTwoStepModel,
    'pls_mlpdecoder': PLS_MLPDecoderModel,
    'pls_bilineardecoder': PLS_BilinearDecoderModel,
    'dynamic_mlp': DynamicMLP,
    'shared_mlp_encoder': SharedMLPEncoderModel,
    'shared_linear_encoder': SharedLinearEncoderModel,
    'shared_transformer': SharedSelfAttentionModel,
    'shared_transformer_cls': SharedSelfAttentionCLSModel,
    'cross_attention': CrossAttentionModel
}




def train_sweep_torch(config, model_type, train_indices, feature_type, connectome_target, dataset, cv_type, cv_obj, outer_fold_idx, device, sweep_id, model_classes, parcellation, hemisphere, omit_subcortical, gene_list, seed, binarize, impute_strategy, sort_genes, null_model):
    """
    Training function for W&B sweeps for deep learning models.
    
    Args:
        config: W&B sweep configuration
        model_type: Type of deep learning model
        feature_type: List of feature dictionaries
        connectome_target: Target connectome type
        cv_type: Type of cross-validation
        outer_fold_idx: Current outer fold index
        inner_fold_splits: List of inner fold data splits
        device: torch device (cuda/cpu)
        sweep_id: Current W&B sweep ID
    
    Returns:
        float: Mean validation loss across inner folds
    """
    feature_str = "+".join(str(k) if v is None else f"{k}_{v}"
                         for feat in feature_type 
                         for k,v in feat.items())
    run_name = f"{model_type}_{feature_str}_{connectome_target}_{cv_type}{seed}_fold{outer_fold_idx}_innerCV" 
    run = wandb.init(
        project="gx2conn",
        name=run_name,
        group=f"sweep_{sweep_id}",
        tags=[
            "inner cross validation",
            f'cv_type_{cv_type}',
            f"fold{outer_fold_idx}",
            f"model_{model_type}",
            f"split_{cv_type}{seed}",
            f'feature_type_{feature_str}',
            f'target_{connectome_target}',
            f"parcellation_{parcellation}",
            f"hemisphere_{hemisphere}",
            f"omit_subcortical_{omit_subcortical}",
            f"gene_list_{gene_list}",
            f"binarize_{binarize}",
            f"impute_strategy_{impute_strategy}",
            f"sort_genes_{sort_genes}",
            f"null_model_{null_model}"
        ],
        reinit=True
    )
    sweep_config = wandb.config
    
    inner_fold_metrics = {'final_train_loss': [], 'final_val_loss': [], 
                          'final_train_pearson': [], 'final_val_pearson': []}

    # Get the appropriate model class
    if model_type not in model_classes:
        raise ValueError(f"Model type {model_type} not supported for W&B sweeps")
    
    ModelClass = model_classes[model_type]

    # Process each inner fold
    for fold_idx, (train_indices, test_indices) in enumerate(cv_obj.split()):
        print(f'Processing inner fold {fold_idx}')
        
        torch._dynamo.reset()  # Clear compiled graph cache
        gc.collect()
        torch.cuda.empty_cache()

        if fold_idx == 0:  # Only run CV on the first inner fold to test more parameters
            train_region_pairs = expand_X_symmetric(np.array(train_indices).reshape(-1, 1)).astype(int)
            test_region_pairs = expand_X_symmetric(np.array(test_indices).reshape(-1, 1)).astype(int)
            train_indices_expanded = np.array([dataset.valid_pair_to_expanded_idx[tuple(pair)] for pair in train_region_pairs])
            test_indices_expanded = np.array([dataset.valid_pair_to_expanded_idx[tuple(pair)] for pair in test_region_pairs])    
    
            # Initialize model dynamically based on sweep config and fit
            if 'pls' in model_type:
                model = ModelClass(**sweep_config, train_indices=train_indices, test_indices=test_indices, region_pair_dataset=dataset).to(device)
            else:
                model = ModelClass(**sweep_config).to(device)
            
            if model_type == 'pls_twostep':
                history = model.fit(dataset, train_indices, test_indices)
                # Store final metrics
                inner_fold_metrics['final_train_loss'].append(history['train_loss'])
                inner_fold_metrics['final_val_loss'].append(history['val_loss'])
                inner_fold_metrics['final_train_pearson'].append(history['train_pearson'])
                inner_fold_metrics['final_val_pearson'].append(history['val_pearson'])
            else: 
                history = model.fit(dataset, train_indices_expanded, test_indices_expanded)
                
                # Log epoch-wise metrics
                for epoch, metrics in enumerate(zip(history['train_loss'], history['val_loss'])):
                    wandb.log({
                        'inner_fold': fold_idx,
                        f'fold{fold_idx}_epoch': epoch,
                        f'fold{fold_idx}_train_loss': metrics[0],
                        f'fold{fold_idx}_val_loss': metrics[1]
                    })
                
                train_dataset = Subset(dataset, train_indices_expanded)
                train_loader = DataLoader(train_dataset, batch_size=512, shuffle=False, pin_memory=True)
                test_dataset = Subset(dataset, test_indices_expanded)
                val_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, pin_memory=True)
            
                predictions, targets = model.predict(train_loader)
                train_pearson = pearsonr(predictions, targets)[0]
                predictions, targets = model.predict(val_loader)
                val_pearson = pearsonr(predictions, targets)[0]

                # Store final metrics
                inner_fold_metrics['final_train_loss'].append(history['train_loss'][-1])
                inner_fold_metrics['final_val_loss'].append(history['val_loss'][-1])
                inner_fold_metrics['final_train_pearson'].append(train_pearson)
                inner_fold_metrics['final_val_pearson'].append(val_pearson)

            # del model
            torch.cuda.empty_cache()
            gc.collect()

    # Log mean metrics across folds
    mean_metrics = {
        'mean_train_loss': np.mean(inner_fold_metrics['final_train_loss']),
        'mean_val_loss': np.mean(inner_fold_metrics['final_val_loss']),
        'mean_train_pearson': np.mean(inner_fold_metrics['final_train_pearson']),
        'mean_val_pearson': np.mean(inner_fold_metrics['final_val_pearson'])
    }
    wandb.log(mean_metrics)
    run.finish()
    
    return mean_metrics['mean_val_loss']




def format_metrics_output(metrics, mode):
    """Format metrics into organized categories for clean printing"""
    # Define metric categories
    global_metrics = ['mse', 'mae', 'r2', 'pearson_r', 'spearman_r', 'geodesic_distance', 'accuracy', 'precision', 'recall', 'f1', 'auc_roc']
    distance_metrics = ['short_r', 'mid_r', 'long_r'] 
    hemispheric_metrics = ['left_hemi_r', 'right_hemi_r', 'inter_hemi_r']
    strength_metrics = ['strong_neg_r', 'weak_r', 'strong_pos_r']
    graph_theory_metrics = ['geodesic_distance']
    network_names = ['Cont', 'Default', 'SalVentAttn', 'Limbic', 'DorsAttn', 'SomMot', 'Vis', 'Subcortical', 'Cerebellum']
    
    print(f"{mode.upper()} METRICS")
    print(f"{'='*50}")

    # Global Metrics
    global_found = {k: v for k, v in metrics.items() if k in global_metrics and not np.isnan(v)}
    if global_found:
        print("GLOBAL:", end=" ")
        metrics_str = []
        for k, v in global_found.items():
            if k in ['mse', 'mae']:
                metrics_str.append(f"{k}={v:.6f}")
            else:
                metrics_str.append(f"{k}={v:.4f}")
        print(", ".join(metrics_str))

    # Distance, Hemispheric and Strength metrics
    distance_found = {k: v for k, v in metrics.items() if k in distance_metrics and not np.isnan(v)}
    if distance_found:
        print("DISTANCE-BASED:", end=" ")
        print(", ".join([f"{k.replace('_r','')}={v:.4f}" for k,v in distance_found.items()]))
    
    hemispheric_found = {k: v for k, v in metrics.items() if k in hemispheric_metrics and not np.isnan(v)}
    if hemispheric_found:
        print("HEMISPHERIC:", end=" ")
        labels = {"left_hemi": "left", "right_hemi": "right", "inter_hemi": "inter"}
        print(", ".join([f"{labels[k.replace('_r','')]}={v:.4f}" for k,v in hemispheric_found.items()]))
    
    strength_found = {k: v for k, v in metrics.items() if k in strength_metrics and not np.isnan(v)}
    if strength_found:
        print("CONNECTION STRENGTH:", end=" ")
        labels = {"strong_neg": "neg", "weak": "weak", "strong_pos": "pos"}
        print(", ".join([f"{labels[k.replace('_r','')]}={v:.4f}" for k,v in strength_found.items()]))

    # Network metrics
    intra_metrics = {k: v for k, v in metrics.items() if k.startswith('intra_network_') and not np.isnan(v)}
    inter_metrics = {k: v for k, v in metrics.items() if k.startswith('inter_network_') and not np.isnan(v)}
    
    if intra_metrics or inter_metrics:
        print("NETWORK CORRELATIONS:")
        print("  NETWORK      INTRA      INTER")
        print("  " + "-" * 30)
        for net in network_names:
            intra_val = metrics.get(f'intra_network_{net}_r')
            inter_val = metrics.get(f'inter_network_{net}_r')
            if not (np.isnan(intra_val) if intra_val is not None else True) or \
               not (np.isnan(inter_val) if inter_val is not None else True):
                intra_str = f"{intra_val:.4f}" if intra_val is not None and not np.isnan(intra_val) else "N/A"
                inter_str = f"{inter_val:.4f}" if inter_val is not None and not np.isnan(inter_val) else "N/A"
                print(f"  {net:<10}  {intra_str:>8}  {inter_str:>8}")

    print(f"{'='*50}" + '\n')



def compute_distance_metrics(y_true, y_pred, distances):
    """Compute distance-based correlation metrics"""
    # Handle both tensor and numpy distances
    if torch.is_tensor(distances):
        distances = distances.cpu().numpy()
    else:
        distances = getattr(distances, 'get', lambda: distances)()

    # Distance range thresholds 
    dist_33, dist_67 = 175/3, 175*2/3
    
    # Create masks
    short_mask = distances <= dist_33
    mid_mask = (distances > dist_33) & (distances <= dist_67)
    long_mask = distances > dist_67

    # Calculate correlations
    metrics = {}
    
    # Convert masks to numpy if needed
    if torch.is_tensor(short_mask):
        short_mask = short_mask.cpu().numpy()
        mid_mask = mid_mask.cpu().numpy() 
        long_mask = long_mask.cpu().numpy()

    # Use numpy sum to count mask elements
    if np.sum(short_mask) > 100:
        metrics['short_r'] = pearsonr(y_true[short_mask], y_pred[short_mask])[0]
    else:
        metrics['short_r'] = np.nan
        
    if np.sum(mid_mask) > 100:
        metrics['mid_r'] = pearsonr(y_true[mid_mask], y_pred[mid_mask])[0]
    else:
        metrics['mid_r'] = np.nan
        
    if np.sum(long_mask) > 100:
        metrics['long_r'] = pearsonr(y_true[long_mask], y_pred[long_mask])[0]
    else:
        metrics['long_r'] = np.nan
        
    return metrics

def compute_strength_metrics(y_true, y_pred):
    """
    Compute correlation metrics based on connection strength ranges.
    
    Args:
        y_true: Ground truth values
        y_pred: Predicted values
        
    Returns:
        Dictionary containing correlations for strong negative (<-0.3), 
        weak (-0.3 to 0.3), and strong positive (>0.3) connections
    """
    # Create masks for different strength ranges
    strong_neg_mask = y_true < -0.3
    weak_mask = (y_true >= -0.3) & (y_true <= 0.3)
    strong_pos_mask = y_true > 0.3
    
    metrics = {}
    
    # Calculate correlations for each range if enough samples exist
    if np.sum(strong_neg_mask) > 100:
        metrics['strong_neg_r'] = pearsonr(y_true[strong_neg_mask], y_pred[strong_neg_mask])[0]
    else:
        metrics['strong_neg_r'] = np.nan
        
    if np.sum(weak_mask) > 100:
        metrics['weak_r'] = pearsonr(y_true[weak_mask], y_pred[weak_mask])[0]
    else:
        metrics['weak_r'] = np.nan
        
    if np.sum(strong_pos_mask) > 100:
        metrics['strong_pos_r'] = pearsonr(y_true[strong_pos_mask], y_pred[strong_pos_mask])[0]
    else:
        metrics['strong_pos_r'] = np.nan
        
    return metrics

def compute_hemispheric_metrics(y_true, y_pred, indices, coords):
    """Compute hemisphere-based correlation metrics"""
    left_true, left_pred = [], []
    right_true, right_pred = [], []
    inter_true, inter_pred = [], []
    
    for idx, (i, j) in enumerate(combinations(indices, 2)):
        # Get x coordinates for both regions
        x_i = coords[i][0]
        x_j = coords[j][0]
        
        # Determine hemisphere based on x coordinates
        if x_i < 0 and x_j < 0:  # Left-left
            left_true.append(y_true[2*idx])
            left_pred.append(y_pred[2*idx])
        elif x_i > 0 and x_j > 0:  # Right-right
            right_true.append(y_true[2*idx])
            right_pred.append(y_pred[2*idx])
        else:  # Inter-hemispheric
            inter_true.append(y_true[2*idx])
            inter_pred.append(y_pred[2*idx])

    metrics = {}
    if len(left_true) > 1:
        metrics['left_hemi_r'] = pearsonr(left_true, left_pred)[0]
    if len(right_true) > 1:
        metrics['right_hemi_r'] = pearsonr(right_true, right_pred)[0]
    if len(inter_true) > 1:
        metrics['inter_hemi_r'] = pearsonr(inter_true, inter_pred)[0]
        
    return metrics
    
def compute_subnetwork_metrics(y_true, y_pred, indices, network_labels, shared_indices=None):
    """
    Compute network-based correlation metrics for both intra and inter-network connections.
    
    Args:
        y_true: True connectivity values
        y_pred: Predicted connectivity values 
        indices: List of region indices
        network_labels: Network label for each region
        shared_indices: Optional list of shared region indices to include connections with
        
    Returns:
        Dictionary with correlations for:
        - Intra-network connections within each network
        - Inter-network connections between networks
    """
    networks = ['Cont', 'Default', 'SalVentAttn', 'Limbic', 
               'DorsAttn', 'SomMot', 'Vis', 'Subcortical', 'Cerebellum']
    
    # Initialize data structures for intra and inter-network connections
    intra_network_data = {net: {'true': [], 'pred': []} for net in networks}
    inter_network_data = {net: {'true': [], 'pred': []} for net in networks}

    # Helper function to process a pair of regions and their values
    def process_region_pair(net_i, net_j, true_val, pred_val):
        # Skip if either network label is not in our list
        if net_i not in networks or net_j not in networks:
            return
            
        # Add to appropriate network data
        if net_i == net_j: # Intra-network connections
            intra_network_data[net_i]['true'].append(true_val)
            intra_network_data[net_i]['pred'].append(pred_val)
        else: # Inter-network connections
            for net in [net_i, net_j]:
                inter_network_data[net]['true'].append(true_val)
                inter_network_data[net]['pred'].append(pred_val)

    # Process connections between indices
    n_pairs = len(list(combinations(indices, 2)))
    for idx, (i, j) in enumerate(combinations(indices, 2)):
        net_i, net_j = network_labels[i], network_labels[j]
        process_region_pair(net_i, net_j, y_true[idx], y_pred[idx])

    # Process connections with shared indices if provided
    if shared_indices is not None:
        offset = n_pairs
        for i_idx, i in enumerate(indices):
            for j_idx, j in enumerate(shared_indices):
                idx = offset + i_idx * len(shared_indices) + j_idx
                if idx < len(y_true):  # Ensure index is within bounds
                    net_i, net_j = network_labels[i], network_labels[j]
                    process_region_pair(net_i, net_j, y_true[idx], y_pred[idx])

    # Calculate metrics
    metrics = {}
    
    # Calculate intra-network correlations
    for network in networks:
        if len(intra_network_data[network]['true']) > 1:
            metrics[f'intra_network_{network}_r'] = pearsonr(
                intra_network_data[network]['true'],
                intra_network_data[network]['pred'])[0]
        else:
            metrics[f'intra_network_{network}_r'] = np.nan
    
    # Calculate inter-network correlations  
    for network in networks:
        if len(inter_network_data[network]['true']) > 1:
            metrics[f'inter_network_{network}_r'] = pearsonr(
                inter_network_data[network]['true'],
                inter_network_data[network]['pred'])[0]
        else:
            metrics[f'inter_network_{network}_r'] = np.nan
            
    return metrics

SyntaxError: invalid syntax (4038994576.py, line 1)

In [9]:
import wandb
import pandas as pd
import numpy as np

# Initialize W&B API
api = wandb.Api()

# Access your project
project = "alexander-ratzan-new-york-university/gx2conn"

# Get all runs from the project
runs = api.runs(project)

# Filter for pls_bilineardecoder runs
pls_bilinear_runs = []
for run in runs:
    # Check if this run is for pls_bilineardecoder model
    if any('pls_bilineardecoder' in tag for tag in run.tags) or 'pls_bilineardecoder' in run.name:
        pls_bilinear_runs.append(run)

print(f"Found {len(pls_bilinear_runs)} pls_bilineardecoder runs")

# # Create a list to store run information
# run_data = []

# for run in pls_bilinear_runs:
#     # Get run summary metrics
#     summary = run.summary
    
#     # Extract Pearson correlation metrics (try different possible metric names)
#     pearson_metrics = {}
#     possible_pearson_keys = [
#         'mean_val_pearson', 'val_pearson', 'pearson_r', 
#         'mean_train_pearson', 'train_pearson', 'final_val_pearson'
#     ]
    
#     for key in possible_pearson_keys:
#         if key in summary:
#             pearson_metrics[key] = summary[key]
    
#     # Use the best available pearson metric (prioritize validation metrics)
#     best_pearson = None
#     metric_name = None
    
#     # Priority order: validation > mean > train > final
#     priority_order = ['mean_val_pearson', 'val_pearson', 'pearson_r', 'final_val_pearson', 'mean_train_pearson', 'train_pearson']
    
#     for metric in priority_order:
#         if metric in pearson_metrics and not np.isnan(pearson_metrics[metric]):
#             best_pearson = pearson_metrics[metric]
#             metric_name = metric
#             break
    
#     if best_pearson is not None:
#         run_info = {
#             'run_id': run.id,
#             'run_name': run.name,
#             'best_pearson': best_pearson,
#             'metric_used': metric_name,
#             'state': run.state,
#             'created_at': run.created_at,
#             'config': dict(run.config),
#             'all_pearson_metrics': pearson_metrics,
#             'tags': run.tags
#         }
#         run_data.append(run_info)

# # Sort by best pearson correlation (descending)
# run_data.sort(key=lambda x: x['best_pearson'], reverse=True)

# print(f"\nTop 10 pls_bilineardecoder models by Pearson correlation:")
# print("=" * 80)

# for i, run_info in enumerate(run_data[:10]):
#     print(f"\nRank {i+1}:")
#     print(f"  Run ID: {run_info['run_id']}")
#     print(f"  Run Name: {run_info['run_name']}")
#     print(f"  Best Pearson: {run_info['best_pearson']:.4f} ({run_info['metric_used']})")
#     print(f"  State: {run_info['state']}")
#     print(f"  Created: {run_info['created_at']}")
    
#     # Show key config parameters
#     config = run_info['config']
#     print(f"  Config highlights:")
#     for key in ['learning_rate', 'batch_size', 'n_components', 'hidden_dim', 'dropout']:
#         if key in config:
#             print(f"    {key}: {config[key]}")
    
#     # Show all pearson metrics for this run
#     print(f"  All Pearson metrics: {run_info['all_pearson_metrics']}")

# # Get the best run for detailed analysis
# if run_data:
#     best_run_info = run_data[0]
#     print(f"\n" + "=" * 80)
#     print(f"BEST MODEL DETAILS:")
#     print(f"=" * 80)
    
#     # Access the actual run object
#     best_run = api.run(f"{project}/{best_run_info['run_id']}")
    
#     print(f"Run ID: {best_run_info['run_id']}")
#     print(f"Run Name: {best_run_info['run_name']}")
#     print(f"Best Pearson: {best_run_info['best_pearson']:.4f}")
#     print(f"State: {best_run_info['state']}")
    
#     print(f"\nFull Configuration:")
#     for key, value in best_run_info['config'].items():
#         print(f"  {key}: {value}")
    
#     print(f"\nAll Summary Metrics:")
#     for key, value in best_run.summary.items():
#         if not key.startswith('_'):  # Skip internal wandb metrics
#             print(f"  {key}: {value}")
    
#     # Download model artifacts if available
#     print(f"\nAvailable Files:")
#     for file in best_run.files():
#         print(f"  - {file.name}")
    
#     # Option to download the best model
#     print(f"\nTo download the best model, run:")
#     print(f"best_run = api.run('{project}/{best_run_info['run_id']}')")
#     print(f"# Download specific files:")
#     print(f"# best_run.file('model.pth').download()")
# else:
#     print("No pls_bilineardecoder runs found with Pearson correlation metrics.")

# # Create a DataFrame for easier analysis
# if run_data:
#     df_data = []
#     for run_info in run_data:
#         row = {
#             'run_id': run_info['run_id'],
#             'run_name': run_info['run_name'],
#             'best_pearson': run_info['best_pearson'],
#             'metric_used': run_info['metric_used'],
#             'state': run_info['state'],
#             'created_at': run_info['created_at']
#         }
#         # Add config parameters as separate columns
#         for key, value in run_info['config'].items():
#             row[f'config_{key}'] = value
        
#         df_data.append(row)
    
#     df = pd.DataFrame(df_data)
#     print(f"\nDataFrame created with {len(df)} runs")
#     print(f"Columns: {list(df.columns)}")
    
#     # Save to CSV for further analysis
#     df.to_csv('pls_bilineardecoder_results.csv', index=False)
#     print(f"Results saved to 'pls_bilineardecoder_results.csv'")

KeyboardInterrupt: 

In [12]:
import wandb
from datetime import datetime, timedelta

api = wandb.Api()

project_path = "alexander-ratzan-new-york-university/gx2conn"

# Get timestamp for 7 days ago (you can change this to 30, etc.)
seven_days_ago = datetime.now() - timedelta(days=7)

# W&B API requires ISO format string
timestamp_filter = {"$gte": seven_days_ago.isoformat()}

# Build the filter
filters = {
    "config.model_type": "pls_bilineardecoder",
    "created_at": timestamp_filter,
    "state": "finished"  # Optional
}

# Fetch the filtered runs (can limit for efficiency)
runs = api.runs(project_path, filters=filters, order="-created_at", per_page=10)

# Print result
for i, run in enumerate(runs):
    print(f"{i + 1}. Run ID: {run.id} | Created: {run.created_at} | Name: {run.name}")